In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount= True)

In [5]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [6]:
def displayImages(images, title1="Original", title2="Augmented", labels=None, augmented_images=None):
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

    fig = plt.figure(figsize=(images.shape[0], images.shape[0]))
    for i in range(images.shape[0]):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        if labels is not None:
            plt.xlabel(class_names[int(labels[i])])
    fig.suptitle(title1, fontsize=16)

    if augmented_images is not None:
        fig2 = plt.figure(2, figsize=(augmented_images.shape[0], augmented_images.shape[0]))
        for i in range(augmented_images.shape[0]):
            plt.subplot(5, 5, i + 1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(augmented_images[i], cmap=plt.cm.binary)
            if labels is not None:
                plt.xlabel(class_names[int(labels[i])])
        fig2.suptitle(title2, fontsize=16)
    plt.show()


def split_indexes(n_classes, n_labeled_per_class, n_validation, labels):
    labels = np.array(labels)
    train_labeled_indexes = []
    train_unlabeled_indexes = []
    validation_indexes = []

    for i in range(n_classes):
        indexes = np.where(labels == i)[0]
        np.random.shuffle(indexes)

        train_labeled_indexes.extend(indexes[:n_labeled_per_class])
        train_unlabeled_indexes.extend(indexes[n_labeled_per_class:-n_validation])
        validation_indexes.extend(indexes[-n_validation:])

    np.random.shuffle(train_unlabeled_indexes)
    np.random.shuffle(train_labeled_indexes)
    np.random.shuffle(validation_indexes)

    return train_labeled_indexes, train_unlabeled_indexes, validation_indexes


def to_tensor_dim(x, source='NHWC', target='NCHW'):
    return x.transpose([source.index(d) for d in target])


def normalise(X):
    mean = np.mean(X, axis=(0, 1, 2))
    std = np.std(X, axis=(0, 1, 2))
    X, mean, std = [np.array(a, np.float32) for a in (X, mean, std)]
    X -= mean
    X *= 1.0 / std
    return X


def random_flip(x):
    if np.random.rand() < 0.6:
        x = x[:, ::-1, :]
    return x.copy()


def pad(x, border=4):
    return np.pad(x, [(border, border), (border, border), (0, 0)], mode='reflect')


def pad_and_crop(x, output_size=(32, 32)):
    x = pad(x, 4)
    h, w = x.shape[:-1]
    new_h, new_w = output_size

    top = np.random.randint(0, h - new_h)
    left = np.random.randint(0, w - new_w)

    x = x[top: top + new_h, left: left + new_w, :]

    return x


def augment(X, K=1):

    # X_augmented = X
    # for k in range(K-1):
    #   X_augmented.hstack(X)
    X_augmented = np.zeros(([K]+list(X.shape)))

    # print("after hstack", X_augmented.shape)
    for k in range(K):
      for i in range(X_augmented[k].shape[0]):
            x = X[i, :]
            x = pad_and_crop(x)
            X_augmented[k, i, :] = random_flip(x)
    return X_augmented


def load_and_augment_data(dataset_name, model_params):
    """
    From datasets.CIFAR10:
        dataset.data: the image as numpy array, shape: (50000, 32, 32, 3)
        dataset.targets: labels of the images as list, len: 50000
    :return:
        augmented_labeled_X: the tensor of augmented labeled images (K=1),
                             size: (n_labeled_per_class * n_classes , 32, 32, 3)
        augmented_unlabeled_X: the tensor of augmented unlabeled images (K=2),
                             size: ((N/10 - n_labeled_per_class - n_validation) * n_classes * K , 32, 32, 3)
        train_labeled_targets: the tensor of labeled targets,
                             size = n_labeled_per_class * n_classes
        train_unlabeled_targets: the tensor of unlabeled targets,
                             size = (N/10 - n_labeled_per_class - n_validation) * n_classes
    """

    # Step 1: Set the model's hyperparameters
    n_classes = model_params["n_classes"]
    n_labeled_per_class = model_params["n_labeled_per_class"]
    n_validation = model_params["n_validation"]
    K = model_params["K"]

    # Step 2: Load the dataset
    if dataset_name == 'CIFAR10':
        dataset = datasets.CIFAR10(root="./datasets", train=True, download=True)
    elif dataset_name == 'SLT10':
        dataset = datasets.STL10(root="./datasets", download=True)
    else:
        raise ValueError("Invalid dataset name")

    # Step 3: Split the indexes
    train_labeled_indexes, train_unlabeled_indexes, validation_indexes = \
        split_indexes(n_classes, n_labeled_per_class, n_validation, dataset.targets)

    # Step 4: Attract the images for training, validation
    train_labeled_images = np.take(dataset.data, train_labeled_indexes, axis=0)
    train_unlabeled_images = np.take(dataset.data, train_unlabeled_indexes, axis=0)
    target_array = np.asarray(dataset.targets)
    train_labeled_targets = np.take(target_array, train_labeled_indexes, axis=0)
    train_unlabeled_targets = np.take(target_array, train_unlabeled_indexes, axis=0)
    validation_images = np.take(dataset.data, validation_indexes, axis=0)
    validation_targets = np.take(target_array, validation_indexes, axis=0)

    # Step 5: Normalise the datasets
    train_labeled_images = normalise(train_labeled_images)
    train_unlabeled_images = normalise(train_unlabeled_images)

    # Step 6: Augment training images
    print("shape",train_unlabeled_images.shape )

    augmented_labeled_X = augment(train_labeled_images, K=1)
    augmented_unlabeled_X = augment(train_unlabeled_images, K=K)
    
    print("shape after", augmented_unlabeled_X.shape )
    
    # Take a look at some of the augmented images
    # displayImages(train_labeled_images[:10], title1="Original-Labeled", title2="Augmented-Labeled",
    #               augmented_images=augmented_labeled_X[:10], labels=train_labeled_targets[:10])
    # n_unlabeled = train_unlabeled_images.shape[0]
    # displayImages(train_unlabeled_images[:10], title1="Original-Unlabeled", title2="Augmented-Unlabeled",
    #               augmented_images=augmented_unlabeled_X[:10], labels=train_unlabeled_targets[:10])
    # displayImages(augmented_unlabeled_X[:10], title1="Augmented-Unlabeled1", title2="Augmented-Unlabeled2",
    #               augmented_images=augmented_unlabeled_X[n_unlabeled:10+n_unlabeled],
    #               labels=train_unlabeled_targets[:10])

    # Step 7: Change the dimension of np.array in oder for it to work in torch
    augmented_labeled_X = to_tensor_dim(augmented_labeled_X.reshape(train_labeled_images.shape))

    augmented_unlabeled_X_zeros = np.zeros(([K]+[augmented_unlabeled_X.shape[1]] + list(augmented_labeled_X.shape[1:])))
    #print("augmented_unlabeled_X_zeros", augmented_unlabeled_X_zeros.shape)
    for k in range(K):
      unlabeled_shape = augmented_unlabeled_X[k].reshape(train_unlabeled_images.shape)
      #print("unlabeled_shape", unlabeled_shape)
      augmented_unlabeled_X_zeros[k] = to_tensor_dim(unlabeled_shape)
    validation_images = to_tensor_dim(validation_images)
    
    return torch.from_numpy(augmented_labeled_X), torch.from_numpy(augmented_unlabeled_X_zeros), \
           torch.from_numpy(train_labeled_targets), torch.from_numpy(train_unlabeled_targets), \
           torch.from_numpy(validation_images), torch.from_numpy(validation_targets)


# Defining Model Parameters and obtaining train and val sets

In [7]:
model_params = {
    "n_classes": 10,
    "n_labeled_per_class": 3000,
    "n_validation": 500,
    "K": 2
}
augmented_labeled_X, augmented_unlabeled_X, train_labeled_targets, train_unlabeled_targets, \
    validation_images, validation_targets = load_and_augment_data('CIFAR10', model_params)

print(augmented_labeled_X.size())
print(augmented_unlabeled_X.size())
print(train_labeled_targets.size())
print(train_unlabeled_targets.size())


# Half the number of labeled data TODO:check this!
n_labeled = augmented_labeled_X.size()[0]
new_n = int(n_labeled/2)
augmented_labeled_X = augmented_labeled_X[:new_n]
print(f"New length: {augmented_labeled_X.size()}")



Extracting ./datasets/cifar-10-python.tar.gz to ./datasets
shape (15000, 32, 32, 3)
shape after (2, 15000, 32, 32, 3)
torch.Size([30000, 3, 32, 32])
torch.Size([2, 15000, 3, 32, 32])
torch.Size([30000])
torch.Size([15000])
New length: torch.Size([15000, 3, 32, 32])


# Splitting data into Batches

In [8]:
batch_size = 10

## Splitting labelled data


In [9]:
permutation = torch.randperm(augmented_labeled_X.size()[0])
labeled_batches = []
for i in range(0,augmented_labeled_X.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = augmented_labeled_X[indices], train_labeled_targets[indices]
        labeled_batches.append((batch_x, batch_y))

# Splitting unlabelled data

In [10]:
#Splitting unlabelled into batches
k_unlabeled_batches = []
for k in range(augmented_unlabeled_X.size()[0]):
  permutation = torch.randperm(augmented_unlabeled_X.size()[1])
  unlabeled_batches = []
  for i in range(0, augmented_unlabeled_X.size()[1], batch_size):
          indices = permutation[i:i+batch_size]
          batch = augmented_unlabeled_X[k][indices]
          unlabeled_batches.append(batch)
  k_unlabeled_batches.append(unlabeled_batches)


In [11]:
k_unlabeled_batches[0][0].size()


torch.Size([10, 3, 32, 32])

K_unlabeled_batches is a tuple of shape [k, num_batches, batch_size, 3, 32, 32]
where k is the number of augmentations.

# Defining Model

In [12]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(classes))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [13]:
def label_guessing(model, data):
   return model(data)

# Example use of label_guessing function

In [14]:
guesses = label_guessing(model_ft, k_unlabeled_batches[0][0].float().to(device))
print(f"shape: {guesses.shape}")
print(guesses)



shape: torch.Size([10, 10])
tensor([[-1.2789, -0.5869, -0.0888,  1.7029, -2.6441, -1.3944,  0.9974,  2.3822,
          1.1978,  0.1573],
        [-0.1398,  0.0064,  0.4604,  0.3415, -0.7673,  1.3372,  1.4283,  0.1526,
          1.1103,  0.7259],
        [-0.0276, -0.7054, -0.6044,  0.3507, -0.2867, -1.2972, -0.0840,  1.6870,
          0.4973, -0.5207],
        [ 0.7743, -0.2185, -0.5104, -0.1310, -1.0116, -1.0090,  2.2560, -0.0497,
          0.1744,  0.0392],
        [ 0.6033, -0.3544,  0.9827,  0.8817, -0.7490, -0.3838,  1.3908,  0.5579,
          0.1137,  1.3341],
        [ 0.1961,  0.1221, -0.2273,  0.0891, -0.8502, -1.3683,  1.8759,  0.4580,
          0.3588,  0.8689],
        [-0.8054,  0.0316,  0.9166,  0.6161, -1.2150,  0.3562,  3.0396,  2.2617,
          0.6982,  1.0428],
        [ 0.5101, -1.1070,  0.5907,  0.9323, -0.5980, -0.9324, -0.5306,  1.8763,
          1.9642,  1.2228],
        [-0.9229, -1.2849, -0.3821,  0.7601, -0.8083, -1.5911, -0.1905, -0.8115,
          0.7612,  

In [44]:
def guess_and_sharpen(augmented_unlabeled_X1, augmented_unlabeled_X2, T=0.5):
    """
    Assume:
      label_guesses: a list of K guesses
    """
    guess1 = label_guessing(model_ft, augmented_unlabeled_X1.float().to(device))
    guess2 = label_guessing(model_ft, augmented_unlabeled_X2.float().to(device))

    # Compute the average predictions across all augmentations
    p = (torch.softmax(guess1, dim=1) + torch.softmax(guess2, dim=1)) / 2

    # Sharpen
    pt = p**(1/T)
    targets_u = pt/pt.sum(dim=1, keepdim=True)
    targets_u = targets_u.detach()

    return targets_u


def mixup(X_hat, data_U1, data_U2, targets_U, alpha):
    data_X = X_hat[0].float().to(device)
    targets_X = X_hat[1]

    targets_X = torch.nn.functional.one_hot(targets_X, num_classes=10).float().to(device)

    batch_size = data_X.size(0)

    # Form W
    all_data = torch.cat([data_X, data_U1.float().to(device), data_U2.float().to(device)], dim=0)
    all_targets = torch.cat([targets_X, targets_U, targets_U], dim=0)

    idx = torch.randperm(all_data.size(0))
    W_data = all_data[idx]
    W_targets = all_targets[idx]

    # Mix it up
    lamda = np.random.beta(alpha, alpha)
    lamda = max(lamda, 1 - lamda)

    data_prime = lamda * all_data + (1-lamda) * W_data
    targets_prime = lamda * all_targets + (1-lamda) * W_targets

    return data_prime, targets_prime


In [45]:
def train(labeled_batches, k_unlabeled_batches):
    # Step 1: Input of the function (Raghav)
      # labeled_batches: List, size: n_batches, each elem: tuple of X(tensor) and targets(tensor)
      # k_unlabeled_batches: [k, num_batches, batch_size, 3, 32, 32] where k is the number of augmentations.

    n_batches = len(labeled_batches)
    alpha = 0.75

    # Step 2: For loop
    for i in range(n_batches):
        targets_u = guess_and_sharpen(k_unlabeled_batches[0][i], k_unlabeled_batches[1][i])
        data_prime, targets_prime = mixup(labeled_batches[i], k_unlabeled_batches[0][i], k_unlabeled_batches[1][i], targets_u, alpha) 

        print(f"data_prime size: {data_prime.size()}")
        print(f"targets_prime size: {targets_prime.size()}")
        
        # data_prime: labeled + u1 + u2, size(3*batch_size, 3, 32, 32)
        # targets_prime: labeled + u1 + u2, size(3*batch_size, 10)

        # Step3: Set the model




In [46]:
train(labeled_batches, k_unlabeled_batches)

data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_prime size: torch.Size([30, 10])
data_prime size: torch.Size([30, 3, 32, 32])
targets_p

#Example Train function

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                #import pdb; pdb.set_trace()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #print(outputs.shape)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / 50000
            epoch_acc = running_corrects.double() / 50000
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

In [ ]:
# import torch.utils.data as data
# batch_size = 10
# labeled_trainloader = load((augmented_labeled_X, train_labeled_targets), batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
# labeled_trainloader

In [ ]:
# labeled_train_iter = iter(labeled_trainloader)

In [ ]:
# inputs_x, targets_x = labeled_train_iter.next()

In [ ]:
# def mix_match(x_batch, y_batch)

In [ ]:
# def train_val_split(labels, n_labeled_per_class):
#     labels = np.array(labels)
#     train_labeled_idxs = []
#     train_unlabeled_idxs = []
#     val_idxs = []

#     for i in range(10):
#         idxs = np.where(labels == i)[0]
#         np.random.shuffle(idxs)
#         train_labeled_idxs.extend(idxs[:n_labeled_per_class])
#         train_unlabeled_idxs.extend(idxs[n_labeled_per_class:-500])
#         val_idxs.extend(idxs[-500:])
#     np.random.shuffle(train_labeled_idxs)
#     np.random.shuffle(train_unlabeled_idxs)
#     np.random.shuffle(val_idxs)

#     return train_labeled_idxs, train_unlabeled_idxs, val_idxs

In [ ]:
# class CIFAR10_labeled(torchvision.datasets.CIFAR10):

#     def __init__(self, root, indexs=None, train=True,
#                  transform=None, target_transform=None,
#                  download=False):
#         super(CIFAR10_labeled, self).__init__(root, train=train,
#                  transform=transform, target_transform=target_transform,
#                  download=download)
#         if indexs is not None:
#             self.data = self.data[indexs]
#             self.targets = np.array(self.targets)[indexs]
#         self.data = transpose(normalise(self.data))

#     def __getitem__(self, index):
#         """
#         Args:
#             index (int): Index
#         Returns:
#             tuple: (image, target) where target is index of the target class.
#         """
#         img, target = self.data[index], self.targets[index]

#         if self.transform is not None:
#             img = self.transform(img)

#         if self.target_transform is not None:
#             target = self.target_transform(target)

#         return img, target
    

# class CIFAR10_unlabeled(CIFAR10_labeled):

#     def __init__(self, root, indexs, train=True,
#                  transform=None, target_transform=None,
#                  download=False):
#         super(CIFAR10_unlabeled, self).__init__(root, indexs, train=train,
#                  transform=transform, target_transform=target_transform,
#                  download=download)
#         self.targets = np.array([-1 for i in range(len(self.targets))])

In [ ]:
# def normalise(x, mean=cifar10_mean, std=cifar10_std):
#     x, mean, std = [np.array(a, np.float32) for a in (x, mean, std)]
#     x -= mean*255
#     x *= 1.0/(255*std)
#     return x

# def transpose(x, source='NHWC', target='NCHW'):
#     return x.transpose([source.index(d) for d in target]) 

# def pad(x, border=4):
#     return np.pad(x, [(0, 0), (border, border), (border, border)], mode='reflect')

# class RandomPadandCrop(object):
#     """Crop randomly the image.
#     Args:
#         output_size (tuple or int): Desired output size. If int, square crop
#             is made.
#     """

#     def __init__(self, output_size):
#         assert isinstance(output_size, (int, tuple))
#         if isinstance(output_size, int):
#             self.output_size = (output_size, output_size)
#         else:
#             assert len(output_size) == 2
#             self.output_size = output_size

#     def __call__(self, x):
#         x = pad(x, 4)

#         h, w = x.shape[1:]
#         new_h, new_w = self.output_size

#         top = np.random.randint(0, h - new_h)
#         left = np.random.randint(0, w - new_w)

#         x = x[:, top: top + new_h, left: left + new_w]

#         return x

# class RandomFlip(object):
#     """Flip randomly the image.
#     """
#     def __call__(self, x):
#         if np.random.rand() < 0.5:
#             x = x[:, :, ::-1]

#         return x.copy()

# class GaussianNoise(object):
#     """Add gaussian noise to the image.
#     """
#     def __call__(self, x):
#         c, h, w = x.shape
#         x += np.random.randn(c, h, w) * 0.15
#         return x

# class ToTensor(object):
#     """Transform the image to tensor.
#     """
#     def __call__(self, x):
#         x = torch.from_numpy(x)
#         return x

In [ ]:
# def get_cifar10(root, n_labeled,
#                  transform_train=None, transform_val=None,
#                  download=True):

#     base_dataset = torchvision.datasets.CIFAR10(root, train=True, download=download)
#     train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(base_dataset.targets, int(n_labeled/10))

#     train_labeled_dataset = CIFAR10_labeled(root, train_labeled_idxs, train=True, transform=transform_train)
#     train_unlabeled_dataset = CIFAR10_unlabeled(root, train_unlabeled_idxs, train=True, transform=TransformTwice(transform_train))
#     val_dataset = CIFAR10_labeled(root, val_idxs, train=True, transform=transform_val, download=True)
#     test_dataset = CIFAR10_labeled(root, train=False, transform=transform_val, download=True)

#     print (f"#Labeled: {len(train_labeled_idxs)} #Unlabeled: {len(train_unlabeled_idxs)} #Val: {len(val_idxs)}")
#     return train_labeled_dataset, train_unlabeled_dataset, val_dataset, test_dataset

In [ ]:
# %cd /content/gdrive/MyDrive/

In [ ]:
# %mkdir DLDS_Data

In [ ]:
# X = datasets.CIFAR10(root='/content/gdrive/MyDrive/DLDS_Data', train=True, download=True)

In [ ]:
# # n_labeled = 100
# # train_labeled_dataset, train_unlabeled_dataset, val_dataset, test_dataset = get_cifar10("/DLDS_Data", n_labeled)

# data_dir = '/content/gdrive/MyDrive/DLDS_Data'
# # image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
# #                                           data_transforms[x])
# #                   for x in ['train', 'val']}
# # dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
# #                                              shuffle=True, num_workers=4)
# #               for x in ['train', 'val']}





In [ ]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root=data_dir , train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root=data_dir , train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# dataloaders = {"train": trainloader}

In [ ]:
# trainloader

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
torch.cuda.is_available()